In [1]:
import numpy as np
import scipy.io as sio
import scipy.linalg as ln

                                                                                Periodic  Boundary Conditions for 2D Box-type domain

In [2]:
NCoarse = np.array([2,2])
NpCoarse = np.prod(NCoarse+1) 
NFine = np.array([8, 8])
NpFine = np.prod(NFine+1)     # Number of "fine-nodes" on τ_h mesh in each direction (1-D array: [x_h, y_h, z_h])
Nepsilon = np.array([4,4])
MFEM= np.array([])

In [3]:
# MFEM = np.ones((16,16))
MFEM = np.array(range(1,82,1)).reshape(9,9)
print(MFEM)

[[ 1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18]
 [19 20 21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34 35 36]
 [37 38 39 40 41 42 43 44 45]
 [46 47 48 49 50 51 52 53 54]
 [55 56 57 58 59 60 61 62 63]
 [64 65 66 67 68 69 70 71 72]
 [73 74 75 76 77 78 79 80 81]]


In [4]:
#LHS node indices
np.arange(0, NCoarse[1]*(NCoarse[0]+1)+1, NCoarse[0]+1)

array([0, 3, 6])

In [5]:
#LHS-ROW-values corresponding to above indices
MFEM[np.arange(0, NCoarse[1]*(NCoarse[0]+1)+1, NCoarse[0]+1),:] 

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [28, 29, 30, 31, 32, 33, 34, 35, 36],
       [55, 56, 57, 58, 59, 60, 61, 62, 63]])

In [6]:
#LHS-ROW-values corresponding to above indices
MFEM[np.arange(0, NCoarse[1]*(NCoarse[0]+1)+1, NCoarse[0]+1),:] 

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [28, 29, 30, 31, 32, 33, 34, 35, 36],
       [55, 56, 57, 58, 59, 60, 61, 62, 63]])

In [7]:
#RHS node indices
np.arange(NCoarse[0], np.prod(NCoarse+1), NCoarse[0]+1)

array([2, 5, 8])

In [8]:
#RHS-ROW-values corresponding to above RHS node-indices
MFEM[np.arange(NCoarse[0], np.prod(NCoarse+1), NCoarse[0]+1),:]

array([[19, 20, 21, 22, 23, 24, 25, 26, 27],
       [46, 47, 48, 49, 50, 51, 52, 53, 54],
       [73, 74, 75, 76, 77, 78, 79, 80, 81]])

In [9]:
#LHS + RHS row values added to the LHS nodal positions
MFEM[np.arange(0, NCoarse[1]*(NCoarse[0]+1)+1, NCoarse[0]+1),:] \
        += MFEM[np.arange(NCoarse[0], np.prod(NCoarse+1), NCoarse[0]+1),:]
MFEM

array([[ 20,  22,  24,  26,  28,  30,  32,  34,  36],
       [ 10,  11,  12,  13,  14,  15,  16,  17,  18],
       [ 19,  20,  21,  22,  23,  24,  25,  26,  27],
       [ 74,  76,  78,  80,  82,  84,  86,  88,  90],
       [ 37,  38,  39,  40,  41,  42,  43,  44,  45],
       [ 46,  47,  48,  49,  50,  51,  52,  53,  54],
       [128, 130, 132, 134, 136, 138, 140, 142, 144],
       [ 64,  65,  66,  67,  68,  69,  70,  71,  72],
       [ 73,  74,  75,  76,  77,  78,  79,  80,  81]])

In [10]:
#LHS node indices
np.arange(0, NCoarse[1] * (NCoarse[0] + 1) + 1, NCoarse[0] + 1)

array([0, 3, 6])

In [11]:
#LHS node indices
np.arange(0, NCoarse[1] * (NCoarse[0] + 1) + 1, NCoarse[0] + 1)

array([0, 3, 6])

In [12]:
#RHS node indices
np.arange(NCoarse[0], np.prod(NCoarse + 1), NCoarse[0] + 1)

array([2, 5, 8])

In [13]:
#RHS-COL-values corresponding to above indices
MFEM[:, np.arange(NCoarse[0], np.prod(NCoarse + 1), NCoarse[0] + 1)]

array([[ 24,  30,  36],
       [ 12,  15,  18],
       [ 21,  24,  27],
       [ 78,  84,  90],
       [ 39,  42,  45],
       [ 48,  51,  54],
       [132, 138, 144],
       [ 66,  69,  72],
       [ 75,  78,  81]])

In [14]:
#LHS + RHS column values added to the LHS nodal positions (LHS boundary points of the box)
MFEM[:, np.arange(0, NCoarse[1] * (NCoarse[0] + 1) + 1, NCoarse[0] + 1)] \
                += MFEM[:, np.arange(NCoarse[0], np.prod(NCoarse + 1), NCoarse[0] + 1)]
MFEM

array([[ 44,  22,  24,  56,  28,  30,  68,  34,  36],
       [ 22,  11,  12,  28,  14,  15,  34,  17,  18],
       [ 40,  20,  21,  46,  23,  24,  52,  26,  27],
       [152,  76,  78, 164,  82,  84, 176,  88,  90],
       [ 76,  38,  39,  82,  41,  42,  88,  44,  45],
       [ 94,  47,  48, 100,  50,  51, 106,  53,  54],
       [260, 130, 132, 272, 136, 138, 284, 142, 144],
       [130,  65,  66, 136,  68,  69, 142,  71,  72],
       [148,  74,  75, 154,  77,  78, 160,  80,  81]])

In [15]:
#BOTTOM-boundary node indices
np.arange(NCoarse[0]+1)

array([0, 1, 2])

In [16]:
#BOTTOM-ROW-values corresponding to above indices
MFEM[np.arange(NCoarse[0]+1), :] 

array([[44, 22, 24, 56, 28, 30, 68, 34, 36],
       [22, 11, 12, 28, 14, 15, 34, 17, 18],
       [40, 20, 21, 46, 23, 24, 52, 26, 27]])

In [17]:
#TOP-boundary node indices
np.arange(NCoarse[1]*(NCoarse[0]+1), np.prod(NCoarse+1))

array([6, 7, 8])

In [18]:
#TOP-ROW-values corresponding to above TOP-nodal indices
MFEM[np.arange(NCoarse[1]*(NCoarse[0]+1), np.prod(NCoarse+1)), :]

array([[260, 130, 132, 272, 136, 138, 284, 142, 144],
       [130,  65,  66, 136,  68,  69, 142,  71,  72],
       [148,  74,  75, 154,  77,  78, 160,  80,  81]])

In [19]:
# BOTTOM + TOP row-values added to the BOTTOM nodal positions (Bbottom boundary of the box)
MFEM[np.arange(NCoarse[0]+1), :] += MFEM[np.arange(NCoarse[1]*(NCoarse[0]+1), np.prod(NCoarse+1)), :]
MFEM

array([[304, 152, 156, 328, 164, 168, 352, 176, 180],
       [152,  76,  78, 164,  82,  84, 176,  88,  90],
       [188,  94,  96, 200, 100, 102, 212, 106, 108],
       [152,  76,  78, 164,  82,  84, 176,  88,  90],
       [ 76,  38,  39,  82,  41,  42,  88,  44,  45],
       [ 94,  47,  48, 100,  50,  51, 106,  53,  54],
       [260, 130, 132, 272, 136, 138, 284, 142, 144],
       [130,  65,  66, 136,  68,  69, 142,  71,  72],
       [148,  74,  75, 154,  77,  78, 160,  80,  81]])

In [20]:
#BOTTOM-boundary nodal indices
np.arange(NCoarse[0] + 1)

array([0, 1, 2])

In [21]:
#BOTTOM-Column-values corresponding to above BOTTOM-boundary nodal-indices
MFEM[:, np.arange(NCoarse[0] + 1)]

array([[304, 152, 156],
       [152,  76,  78],
       [188,  94,  96],
       [152,  76,  78],
       [ 76,  38,  39],
       [ 94,  47,  48],
       [260, 130, 132],
       [130,  65,  66],
       [148,  74,  75]])

In [22]:
# TOP-boundary nodal indices
np.arange(NCoarse[1] * (NCoarse[0] + 1), np.prod(NCoarse + 1))

array([6, 7, 8])

In [23]:
# #TOP-Column-values corresponding to above TOP-boundary nodal-indices
MFEM[:, np.arange(NCoarse[1] * (NCoarse[0] + 1), np.prod(NCoarse + 1))]

array([[352, 176, 180],
       [176,  88,  90],
       [212, 106, 108],
       [176,  88,  90],
       [ 88,  44,  45],
       [106,  53,  54],
       [284, 142, 144],
       [142,  71,  72],
       [160,  80,  81]])

In [24]:
# BOTTOM + TOP column-values added to the BOTTOM nodal positions (Bbottom boundary of the box)
MFEM[:, np.arange(NCoarse[0] + 1)] += MFEM[:, np.arange(NCoarse[1] * (NCoarse[0] + 1), np.prod(NCoarse + 1))]
MFEM

array([[656, 328, 336, 328, 164, 168, 352, 176, 180],
       [328, 164, 168, 164,  82,  84, 176,  88,  90],
       [400, 200, 204, 200, 100, 102, 212, 106, 108],
       [328, 164, 168, 164,  82,  84, 176,  88,  90],
       [164,  82,  84,  82,  41,  42,  88,  44,  45],
       [200, 100, 102, 100,  50,  51, 106,  53,  54],
       [544, 272, 276, 272, 136, 138, 284, 142, 144],
       [272, 136, 138, 136,  68,  69, 142,  71,  72],
       [308, 154, 156, 154,  77,  78, 160,  80,  81]])

In [25]:
# Abandoning TOP-boundary nodal indices 
np.arange(NCoarse[1] * (NCoarse[0] + 1), NpCoarse)

array([6, 7, 8])

In [26]:
#Abandoning RHS-boundary nodal indices without overlapping the TOP-Right corner point
np.arange(NCoarse[0], NpCoarse - 1, NCoarse[0] + 1)

array([2, 5])

In [27]:
# All the abandoning boundary points
fixed_DoF = np.concatenate((np.arange(NCoarse[1] * (NCoarse[0] + 1), NpCoarse), np.arange(NCoarse[0], NpCoarse - 1, NCoarse[0] + 1)))
fixed_DoF

array([6, 7, 8, 2, 5])

In [29]:
# Rest of the nodal indices 
free_DoF = np.setdiff1d(np.arange(NpCoarse), fixed_DoF)
free_DoF 

array([0, 1, 3, 4])

In [30]:
# Row-values of all the used nodal-points when Periodic B.C. applied + All other nodal column values
MFEM[free_DoF]

array([[656, 328, 336, 328, 164, 168, 352, 176, 180],
       [328, 164, 168, 164,  82,  84, 176,  88,  90],
       [328, 164, 168, 164,  82,  84, 176,  88,  90],
       [164,  82,  84,  82,  41,  42,  88,  44,  45]])

In [31]:
# The column values of only the used nodas extracted from above matix. (i.e. All the nadal values after Perioc B.C. are applied)
M_Free_DoF = MFEM[free_DoF][:, free_DoF]
M_Free_DoF 

array([[656, 328, 328, 164],
       [328, 164, 164,  82],
       [328, 164, 164,  82],
       [164,  82,  82,  41]])

                                                                                        Periodic Boundary Conditions for 1D domain

In [32]:
NCoarse = np.array([3])
NpCoarse = np.prod(NCoarse+1) 
NpCoarse

4

In [33]:
D1 = np.array(range(1,17,1)).reshape(4,4)
D1

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [34]:
#LHS node index
D1[0]

array([1, 2, 3, 4])

In [35]:
# RHS node index
D1[-1]

array([13, 14, 15, 16])

In [36]:
D1[0] += D1[-1]
D1

array([[14, 16, 18, 20],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [37]:
np.arange(NpCoarse)

array([0, 1, 2, 3])

In [38]:
# Abandoning last index
free_DoF = np.setdiff1d(np.arange(NpCoarse-1), D1[-1])
free_DoF 

array([0, 1, 2])

In [39]:
D1_Free_DoF = D1[free_DoF][:, free_DoF]
D1_Free_DoF 

array([[14, 16, 18],
       [ 5,  6,  7],
       [ 9, 10, 11]])